In [1]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

In [2]:
#stock analysis 

start = datetime.datetime(2020,2,1)
end = datetime.datetime(2020,10,11)

In [3]:
stock = [] 
i = 'AAL'
stock = yf.download(i, start = start, end = end, progress = False) 

In [4]:
stock

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-02-03,26.930000,27.670000,26.740000,27.160000,27.059999,7630400
2020-02-04,27.680000,28.610001,27.549999,28.430000,28.430000,10832900
2020-02-05,28.959999,29.240000,28.549999,29.100000,29.100000,8959700
2020-02-06,29.260000,29.450001,28.139999,28.299999,28.299999,8335800
2020-02-07,28.059999,28.549999,27.780001,28.379999,28.379999,6231800
...,...,...,...,...,...,...
2020-10-05,13.090000,13.200000,12.750000,13.120000,13.120000,53465500
2020-10-06,13.280000,13.440000,12.460000,12.530000,12.530000,85252400
2020-10-07,12.980000,13.180000,12.810000,13.070000,13.070000,61274200


In [5]:
import datetime

In [6]:
stock.reset_index(inplace=True)


In [7]:
stock

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-02-03,26.930000,27.670000,26.740000,27.160000,27.059999,7630400
1,2020-02-04,27.680000,28.610001,27.549999,28.430000,28.430000,10832900
2,2020-02-05,28.959999,29.240000,28.549999,29.100000,29.100000,8959700
3,2020-02-06,29.260000,29.450001,28.139999,28.299999,28.299999,8335800
4,2020-02-07,28.059999,28.549999,27.780001,28.379999,28.379999,6231800
...,...,...,...,...,...,...,...
170,2020-10-05,13.090000,13.200000,12.750000,13.120000,13.120000,53465500
171,2020-10-06,13.280000,13.440000,12.460000,12.530000,12.530000,85252400
172,2020-10-07,12.980000,13.180000,12.810000,13.070000,13.070000,61274200
173,2020-10-08,13.330000,13.680000,12.730000,13.160000,13.160000,88851700


In [8]:
stock['year'] = pd.DatetimeIndex(stock['Date']).year
stock['month'] = pd.DatetimeIndex(stock['Date']).month
stock['day'] = pd.DatetimeIndex(stock['Date']).day

In [9]:
# lets try to predict the adjusted close with just the day and month

In [10]:
stock_copy = stock 

stock = stock[['month', 'day', 'Adj Close']]

#simple pipeline
import sklearn

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
y = stock['Adj Close'] 
x = stock.drop('Adj Close', axis = 1)


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 24)

from sklearn.ensemble import RandomForestRegressor

param_grid = { 
    'n_estimators': [200, 500, 1000, 2000]
}
from sklearn.model_selection import GridSearchCV

    




In [11]:
model = GridSearchCV(RandomForestRegressor(random_state = 42), param_grid, cv = 3)

    
model.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
             param_grid={'n_estimators': [200, 500, 1000, 2000]})

In [12]:
y_pred = model.predict(X_test)
    
print('\nModel Score: ',model.score(X_test, y_test),'\n')
    


Model Score:  0.9728634793461225 



In [13]:
# lets see how accurate it actually is 

In [14]:
import seaborn as sns 

df = pd.DataFrame(X_test)
df['Actual'] = y_test
df['Predicted'] = y_pred 
df.head()

,month,day,Actual,Predicted
108,7,8,11.990000,12.15825
158,9,17,13.630000,12.80150
93,6,16,17.030001,16.64380
150,9,4,13.610000,12.73410
57,4,24,10.310000,10.32120


In [15]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_pred, y_test)

0.6635500613621302

In [16]:
from sklearn.metrics import mean_absolute_percentage_error as MAPE

In [17]:
MAPE(y_pred, y_test)

0.046044296872695255

In [18]:
#Percent Difference -  (Value1 - Value2)/Value1   * 100 
#So if I want to see the percent my investment changed relative to the price I bought it at 
bought_price = 10
predicted_percent = ((df['Predicted'] - bought_price)/df['Predicted']) * 100

df['Predicted Percent Change'] = predicted_percent
predicted_percent = ((df['Actual'] - bought_price)/df['Actual'])* 100

df['Actual Percent Change'] = predicted_percent
df.head()

#lets see on average how much the stock is off 

,month,day,Actual,Predicted,Predicted Percent Change,Actual Percent Change
108,7,8,11.990000,12.15825,17.751320,16.597163
158,9,17,13.630000,12.80150,21.884155,26.632429
93,6,16,17.030001,16.64380,39.917568,41.280096
150,9,4,13.610000,12.73410,21.470696,26.524612
57,4,24,10.310000,10.32120,3.112042,3.006793


In [19]:
mean_absolute_error(df['Predicted Percent Change'], df['Actual Percent Change'])

#looks like it will on average be off by 3%

3.3364317518449162

In [21]:
df.to_excel('Stock_info.xlsx')